In [1]:
#imports
import pandas as pd
import numpy as np
from dateutil.parser import parse

In [2]:
# import and read data
samples = pd.read_excel("/Users/inesschwartz/Desktop/Thesis/tables_soil_database/AmostrasAngolaTerrario.xlsx")
analyses = pd.read_excel("/Users/inesschwartz/Desktop/Thesis/tables_soil_database/Horizontes Analises.xlsx")
morphology = pd.read_excel("/Users/inesschwartz/Desktop/Thesis/tables_soil_database/Horizontes_Morfologia.xlsx")
profile_loc = pd.read_excel("/Users/inesschwartz/Desktop/Thesis/tables_soil_database/Perfis_local.xlsx")
soil_profile = pd.read_excel("/Users/inesschwartz/Desktop/Thesis/tables_soil_database/Perfis_solo.xlsx")
elemental_analyses = pd.read_excel("/Users/inesschwartz/Desktop/Thesis/tables_soil_database/Data XRF Angola_inicial.xlsx")
#soil_type = pd.read_excel("/Users/inesschwartz/Desktop/Thesis/tables_soil_database/Perfis_solo.xlsx")

# samples table

In [3]:
samples = pd.read_excel("/Users/inesschwartz/Desktop/Thesis/tables_soil_database/AmostrasAngolaTerrario.xlsx")
# Rename columns in the 'dat' DataFrame
samples.rename(columns={
    'Registo': 'sample_id',
    'Nº Campo': 'site_info_id',
    'Ano': 'year',
    'Perfil': 'profile_id',
    'Campanha': 'campaign',
    'Colónia_Pais': 'country',
    'Distrito': 'district',
    'AmostraCrivada': 'sample_sifted',
    'AmostraNaoCrivada': 'sample_not_sifted',
    'Prateleira': 'shelf',
    'Sala': 'room'
}, inplace=True)


# Replace nulls with the string 'null'
samples_filled = samples.fillna('null')

# Filter out and print duplicated samples (including the first occurrence)
duplicated_values = samples['sample_id'][samples['sample_id'].duplicated()].unique()
print(duplicated_values)

duplicated_sites = samples['site_info_id'][samples['site_info_id'].duplicated()].unique()
print(duplicated_sites)

samples['site_info_id'] = samples['site_info_id'].astype(str)

year_clean = samples['year'] = pd.to_datetime(samples['year'], format='%Y').dt.year

# remove non-utf-8 characters in dataframa
#samples_filled = samples_filled.applymap(lambda x: x.encode('utf-8', 'ignore').decode('utf-8') if isinstance(x, str) else x)

# # strip leading/trailing whitespace
# str_cols = samples_filled.select_dtypes(include=['object', 'string']).columns
# for col in str_cols:
#     samples_filled[col] = samples_filled[col].str.strip()

# drop columns i won't be using
samples_filled = samples_filled.drop('Obs', axis=1)

samples_clean = samples_filled
samples_clean.head()
#samples_clean.to_csv("/Users/inesschwartz/Desktop/Thesis/tables_clean/samples_clean.csv", index=False)

[]
['172' '173' '175' ... '1072' '1077' '1078']


,sample_id,site_info_id,year,profile_id,campaign,country,district,sample_sifted,Província,sample_not_sifted,shelf,room
0,630,172,1946.0,139,MA/46,Angola,Huambo,X,Huambo,null,1,22
1,631,173,1946.0,139,MA/46,Angola,Huambo,X,Huambo,null,1,22
2,632,174,1946.0,139,MA/46,Angola,Huambo,X,Huambo,null,1,22
3,633,175,1946.0,139,MA/46,Angola,Huambo,X,Huambo,null,1,22
4,687,1034,1946.0,208,MA/46,Angola,Huambo,X,Huambo,null,1,22


In [4]:
## reorder tables to fit schema
# SAMPLES (
#     sample_id,          -- int, PK
#     sample
#     profile,         -- FK
#     lab_info_id,        -- FK
#     Horizon_number,     -- FK, int
#     Shelf,              -- VARCHAR
#     Room,               -- VARCHAR
#     Year                -- int
# )

#drop from samples_clean: campaign, country, province, sample_not_sifted

# Drop unnecessary columns
samples_clean = samples_filled.drop(columns=[
    'campaign', 'country', 'Província', 'sample_not_sifted', 'sample_sifted'
])

# Add missing columns
samples_clean['lab_info_id'] = pd.NA
samples_clean['horizon_id'] = pd.NA  # only if you don't already have it — remove if it exists


# Rename for consistent schema (only if needed)
samples_clean = samples_clean.rename(columns={
    'profile': 'profile',
    'shelf': 'shelf',
    'room': 'Room',
    'year': 'year'
})

# Reorder columns to match SAMPLES schema
samples_ready = samples_clean[[
    'sample_id',
    'site_info_id',
    'profile_id',
    'horizon_id',
    'shelf',
    'Room',
    'year'
]]

samples_ready['horizon_id']=samples_ready['horizon_id'].astype(str)


# Preview the result
samples_ready.head()
#samples_ready.to_csv("/Users/inesschwartz/Desktop/Thesis/tables_clean/samples_clean.csv", index=False)


/var/folders/tp/79mdnyy56_xc3g1jvp9wf4_80000gn/T/ipykernel_8876/2902848211.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  samples_ready['horizon_id']=samples_ready['horizon_id'].astype(str)


,sample_id,site_info_id,profile_id,horizon_id,shelf,Room,year
0,630,172,139,<NA>,1,22,1946.0
1,631,173,139,<NA>,1,22,1946.0
2,632,174,139,<NA>,1,22,1946.0
3,633,175,139,<NA>,1,22,1946.0
4,687,1034,208,<NA>,1,22,1946.0


In [5]:
samples_ready['shelf'] = samples_ready['shelf'].astype("string")


/var/folders/tp/79mdnyy56_xc3g1jvp9wf4_80000gn/T/ipykernel_8876/1149372123.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  samples_ready['shelf'] = samples_ready['shelf'].astype("string")


In [6]:
import pandas as pd
import datetime

# Expected types
expected_types = {
    'sample_id': float,
    'site_info_id': str,
    'profile_id': str,
    'horizon_id': float,
    'shelf': str,
    'year': 'year'  # Special handling for year as date
}

# Function to check if a column is a year (date or int-like)
def is_year_column(series):
    if pd.api.types.is_datetime64_any_dtype(series):
        return all(series.dt.year.between(1900, datetime.datetime.now().year))
    elif pd.api.types.is_integer_dtype(series):
        return all((series >= 1900) & (series <= datetime.datetime.now().year))
    return False

# Check data types
for column, expected_type in expected_types.items():
    if column not in samples_ready.columns:
        print(f"❌ Column '{column}' not found in DataFrame.")
        continue

    actual_dtype = samples_ready[column].dtype

    if expected_type == 'year':
        if is_year_column(samples_ready[column]):
            print(f"✅ Column '{column}' is a valid year.")
        else:
            print(f"❌ Column '{column}' is not a valid year.")
    elif expected_type == float:
        if pd.api.types.is_float_dtype(samples_ready[column]):
            print(f"✅ Column '{column}' is float.")
        else:
            print(f"❌ Column '{column}' is not float, found {actual_dtype}.")
    elif expected_type == int:
        if pd.api.types.is_integer_dtype(samples_ready[column]):
            print(f"✅ Column '{column}' is int.")
        else:
            print(f"❌ Column '{column}' is not int, found {actual_dtype}.")
    elif expected_type == str:
        if pd.api.types.is_string_dtype(samples_ready[column]):
            print(f"✅ Column '{column}' is string.")
        else:
            print(f"❌ Column '{column}' is not string, found {actual_dtype}.")


❌ Column 'sample_id' is not float, found int64.
✅ Column 'site_info_id' is string.
✅ Column 'profile_id' is string.
❌ Column 'horizon_id' is not float, found object.
✅ Column 'shelf' is string.
❌ Column 'year' is not a valid year.


In [7]:
import pandas as pd
import datetime

# Expected types and target conversions
expected_types = {
    'sample_id': float,
    'site_info_id': str,
    'profile_id': str,
    'horizon_id': object,
    'shelf': object,
    'year': 'year'  # special case
}

# Function to check if a column is a valid year
def is_year_column(series):
    if pd.api.types.is_datetime64_any_dtype(series):
        return all(series.dt.year.between(1900, datetime.datetime.now().year))
    elif pd.api.types.is_integer_dtype(series):
        return all((series >= 1900) & (series <= datetime.datetime.now().year))
    return False

# Loop through columns to check and convert
for column, expected_type in expected_types.items():
    if column not in samples_ready.columns:
        print(f"❌ Column '{column}' not found in DataFrame.")
        continue

    try:
        if expected_type == 'year':
            # Convert to datetime if needed
            if not pd.api.types.is_datetime64_any_dtype(samples_ready[column]):
                samples_ready[column] = pd.to_datetime(samples_ready[column], errors='coerce')
            # Extract year as integer
            samples_ready[column] = samples_ready[column].dt.year
            print(f"✅ Column '{column}' converted to year (int).")

        elif expected_type == float:
            samples_ready[column] = samples_ready[column].astype(float)
            print(f"✅ Column '{column}' converted to float.")

        elif expected_type == int:
            samples_ready[column] = samples_ready[column].astype(int)
            print(f"✅ Column '{column}' converted to int.")

        elif expected_type == str:
            samples_ready[column] = samples_ready[column].astype(str)
            print(f"✅ Column '{column}' converted to string.")

    except Exception as e:
        print(f"❌ Failed to convert column '{column}' to {expected_type}: {e}")


✅ Column 'sample_id' converted to float.
✅ Column 'site_info_id' converted to string.
✅ Column 'profile_id' converted to string.
✅ Column 'year' converted to year (int).


/var/folders/tp/79mdnyy56_xc3g1jvp9wf4_80000gn/T/ipykernel_8876/332639328.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  samples_ready[column] = samples_ready[column].astype(float)
/var/folders/tp/79mdnyy56_xc3g1jvp9wf4_80000gn/T/ipykernel_8876/332639328.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  samples_ready[column] = samples_ready[column].astype(str)
/var/folders/tp/79mdnyy56_xc3g1jvp9wf4_80000gn/T/ipykernel_8876/332639328.py:46: SettingWithCopyWarning: 
A value is trying to be set on a c

## ANALYSES table

In [8]:
analyses.rename(columns={
    'Amostra': 'sample_id',
    'Morfo_id': 'horizon_id',
    'Analise-id': 'analysis_id',
    'PERFIL': 'profile_id',
    'LS': 'upper_limit',
    'LI': 'lower_limit',
    'EG': 'EG',
    'AG': 'thick_clay',
    'AF': 'fine_clay',
    'Argila': 'clay',
    'L': 'silt',
    'Gesso': 'gypsum',
    'Fe livre': 'free_iron',
    'CO':'organic_carbon',
    'N total': 'total_N',
    'MO': 'OM',
    'Soma de bases': 'exchangable_bases_sum',
    'CTC': 'CEC',
    'Cloretos':'chlorides',
    'Sulfatos':'sulfates',
    'Condutividade':'conductivity',
    'Sódio solúvel':'soluble_sodium',
    'P205 total': 'P205',
    'pH (H2O)':'pH_H2O',
    'pH(KCL)':'pH_KCL',
    'MO': 'organic_material'
}, inplace=True)

# Replace nulls with the string 'null'
analyses_filled = analyses.fillna('null')

# Filter out and print duplicated samples (including the first occurrence)
duplicated_values = analyses['sample_id'][analyses['sample_id'].duplicated()].unique()
print(duplicated_values)

# Drop columns not needed
analyses_drop = analyses.drop([
    'EqMol (SiO2)', 'EqMol(Al2O3)', 'EqMol(Fe2O3)', 'SiO2/Al2O3', 
    'SiO2/Fe2O3', 'SiO2/R2O3', 'Fe2O3/Al2O3', 'FE2O3_TARG', 
    'FE2O3_LARG', 'CEC_ARG', 'GR', 'ID', 'COD_PROV'
], axis=1)

# Add a new Primary Key ID column starting from 1
analyses_drop.insert(0, 'lab_sample_id', range(1, len(analyses_drop) + 1))

# Final cleaned dataframe
analyses2 = analyses_drop


#preview
analyses2.head()


[10337.    nan 11608.  3497.  3498.  2923.  2924.  2925.  2926.  2927.
  2928.  2929.  2930. 16355. 16356.   419.   420. 15682. 13817. 13965.
  3418.  9230.  8639.  3529.]


,lab_sample_id,sample_id,horizon_id,analysis_id,profile_id,NA,upper_limit,lower_limit,EG,thick_clay,...,V,chlorides,sulfates,conductivity,soluble_sodium,"Min_<0,002","Min_0,05-0,02","Min_0,2-0,05","Min_2-0,2",Confirmar
0,1,10999.0,B_101/62_1_1,B_101/62_1_1,101/62,1,0.0,11.0,NaN,61.700001,...,23.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
1,2,11000.0,B_101/62_2_1,B_101/62_2_1,101/62,2,11.0,28.0,NaN,52.799999,...,10.6,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2,3,11001.0,B_101/62_3_1,B_101/62_3_1,101/62,3,28.0,54.0,NaN,42.500000,...,5.6,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
3,4,11002.0,B_101/62_4_1,B_101/62_4_1,101/62,4,55.0,85.0,NaN,42.599998,...,8.8,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
4,5,11003.0,B_101/62_5_2,B_101/62_5_21,101/62,5,90.0,120.0,NaN,36.799999,...,7.7,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN


In [9]:
# ANALYSES


# Reorder columns to match SAMPLES schema
# Define the desired final column order
final_columns = [
    'lab_sample_id',
    'sample_id',
    'minerology_id',
    'EG',
    'thick_clay',
    'fine_clay',
    'silt',
    'clay',
    'Eq_Hum',
    'atm_1/3',
    'atm_15',
    'CACO3',
    'gypsum',
    'free_iron',
    'organic_carbon',
    'total_N',
    'P205',
    'organic_material',
    'pH_H2O',
    'pH_KCL',
    'Ca++',
    'Mg++',
    'Na+',
    'K+',
    'exchangable_bases_sum',
    'CEC',
    'V',
    'conductivity',
    'soluble_sodium',
    'Min_<0,002',
    'Min_0,05-0,02',
    'Min_0,2-0,05',
    'Min_2-0,2',
]


# Add missing columns with NaN values
for col in final_columns:
    if col not in analyses2.columns:
        analyses2[col] = pd.NA  # or np.nan if you prefer

# Reorder columns
analyses_ready = analyses2[final_columns]

# Preview first 20 rows
analyses_ready.head(20)

# Export to CSV
#analyses_ready.to_csv("/Users/inesschwartz/Desktop/Thesis/tables_clean/analyses_clean.csv", index=False)

,lab_sample_id,sample_id,minerology_id,EG,thick_clay,fine_clay,silt,clay,Eq_Hum,atm_1/3,...,K+,exchangable_bases_sum,CEC,V,conductivity,soluble_sodium,"Min_<0,002","Min_0,05-0,02","Min_0,2-0,05","Min_2-0,2"
0,1,10999.0,<NA>,NaN,61.700001,32.8,0.2,5.3,4.600000,NaN,...,0.03,NaN,1.83,23.000000,NaN,0.0,NaN,NaN,NaN,NaN
1,2,11000.0,<NA>,NaN,52.799999,35.1,0.7,11.4,6.400000,NaN,...,0.03,NaN,1.98,10.600000,NaN,0.0,NaN,NaN,NaN,NaN
2,3,11001.0,<NA>,NaN,42.500000,46.2,0.2,11.1,6.300000,NaN,...,0.01,NaN,1.62,5.600000,NaN,0.0,NaN,NaN,NaN,NaN
3,4,11002.0,<NA>,NaN,42.599998,41.8,0.2,15.4,5.200000,NaN,...,0.01,NaN,0.91,8.800000,NaN,0.0,NaN,NaN,NaN,NaN
4,5,11003.0,<NA>,NaN,36.799999,47.5,1.2,14.5,7.300000,NaN,...,0.01,NaN,1.04,7.700000,NaN,0.0,NaN,NaN,NaN,NaN
5,6,11004.0,<NA>,NaN,45.099998,41.8,1.4,11.7,0.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
6,7,11011.0,<NA>,1.0,15.300000,48.5,17.6,18.5,19.700001,NaN,...,0.18,NaN,4.49,32.500000,NaN,0.0,NaN,NaN,NaN,NaN
7,8,11012.0,<NA>,3.0,18.600000,42.3,17.8,21.3,19.600000,NaN,...,0.07,NaN,2.96,20.600000,NaN,0.0,NaN,NaN,NaN,NaN
8,9,11013.0,<NA>,7.0,12.000000,36.8,22.1,29.1,21.700001,NaN,...,0.04,NaN,3.02,16.900000,NaN,0.0,NaN,NaN,NaN,NaN
9,10,11014.0,<NA>,21.0,11.000000,34.8,21.6,32.6,21.600000,NaN,...,0.03,NaN,3.11,19.299999,NaN,0.0,NaN,NaN,NaN,NaN


## joining elemental analyses to analyses (still not correct)


In [10]:
# Rename 'Lab. Code' to 'sample_id'
elemental_analyses.rename(columns={'Lab. Code': 'sample_id'}, inplace=True)

# Temporarily convert to string to do the replacements
sample_id_cleaned = elemental_analyses['sample_id'].astype(str)
sample_id_cleaned = sample_id_cleaned.str.replace('C-', '', regex=False)
sample_id_cleaned = sample_id_cleaned.str.replace(' MNL', '', regex=False)

# Then convert to numeric (int or float, depending on your data)
elemental_analyses['sample_id'] = pd.to_numeric(sample_id_cleaned, errors='coerce')

elemental_analyses.head()

,sample_id,Soil Profile,Code,Code.1,Depht,Sampling Date,Mg,Al,Si,P,...,Ta,W,Pt,Au,Hg,Tl,Pb,Bi,Th,U
0,16105,PS7C/55,A-159C,S-1159,0.00-0.11,1965-06-07,0.0,50563.666667,383607.333333,606.333333,...,NaN,NaN,6.0,NaN,5.666667,4.0,NaN,NaN,NaN,NaN
1,16217,P83C/65,A-171C,S-1171,0.00-0.15,1965-07-09,1168.0,45979.000000,393787.666667,609.000000,...,NaN,NaN,NaN,NaN,6.666667,3.0,NaN,3.0,NaN,NaN
2,16033,PSOC/65,A.87C,S-1087,0.00-0.13,1965-07-19,0.0,49541.333333,390160.000000,694.333333,...,21.0,NaN,NaN,NaN,7.666667,NaN,10.666667,NaN,11.0,NaN
3,16252,P115C/65,A-312C,S-1312,0.00-0.15,1965-07-27,0.0,77839.666667,358230.000000,601.666667,...,28.0,10.0,NaN,NaN,9.666667,3.0,NaN,NaN,10.5,NaN
4,16231,P276C/65,A-291C,S-1291,0.00-0.12,1965-08-08,0.0,41215.000000,392816.333333,702.666667,...,20.0,NaN,NaN,NaN,7.333333,NaN,NaN,NaN,NaN,NaN


In [11]:
elemental_analyses.rename(columns={
    'Code': 'field_sample_code'
}, inplace=True)

In [12]:
print (analyses_ready['sample_id'].dtype)
print (elemental_analyses['sample_id'].dtype)

float64
int64


In [13]:
elemental_analyses['sample_id'] = elemental_analyses['sample_id'].astype(float)


In [14]:
# Check which sample_ids in elemental_analyses are NOT in analyses_ready
missing_ids = elemental_analyses[~elemental_analyses['sample_id'].isin(analyses_ready['sample_id'])]

# Display them
print("Sample IDs in elemental_analyses not found in analyses_ready:")
print(missing_ids['sample_id'].unique())


Sample IDs in elemental_analyses not found in analyses_ready:
[16105. 16217. 16033. 16231. 16208. 16274. 16225. 15731. 16078. 15786.
 15693. 15678. 15959. 16372. 16418. 16459. 16477. 15498. 15437. 12749.
 13109. 14337. 14319. 15484. 15508. 15614. 15580. 15582. 15606. 15463.
 14496. 11343. 17892. 16976. 17269. 18248. 16946. 17686. 17728. 17415.
 17642. 18212. 17016. 17282. 18429. 17340. 18824. 18448.  8721.  7313.
  5502.  5392.  7317.  8626.  7193.  7254.  7293.  8240. 15397. 14403.
 14983. 16126. 15059. 15322. 15268. 14808. 15106.]


In [15]:
merged1= analyses_ready.merge(elemental_analyses, on='sample_id', how='left')
merged1.head()


,lab_sample_id,sample_id,minerology_id,EG,thick_clay,fine_clay,silt,clay,Eq_Hum,atm_1/3,...,Ta,W,Pt,Au,Hg,Tl,Pb,Bi,Th,U
0,1,10999.0,<NA>,NaN,61.700001,32.8,0.2,5.3,4.6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,11000.0,<NA>,NaN,52.799999,35.1,0.7,11.4,6.4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,11001.0,<NA>,NaN,42.500000,46.2,0.2,11.1,6.3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,11002.0,<NA>,NaN,42.599998,41.8,0.2,15.4,5.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,11003.0,<NA>,NaN,36.799999,47.5,1.2,14.5,7.3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
print(analyses_ready[analyses_ready['sample_id'] == 6940])
print(elemental_analyses[elemental_analyses['sample_id'] == 6940])


      lab_sample_id  sample_id minerology_id  EG  thick_clay  fine_clay  silt  \
1361           1362     6940.0          <NA> NaN        31.1       43.8   2.7   

      clay  Eq_Hum  atm_1/3  ...  K+  exchangable_bases_sum  CEC   V  \
1361  22.7    18.9      NaN  ... NaN                    NaN  NaN NaN   

      conductivity  soluble_sodium  Min_<0,002  Min_0,05-0,02  Min_0,2-0,05  \
1361           NaN             0.0         NaN            NaN           NaN   

      Min_2-0,2  
1361        NaN  

[1 rows x 33 columns]
    sample_id Soil Profile field_sample_code Code.1      Depht Sampling Date  \
80     6940.0      P134/59          A-320/59  S-492  1.30-1.70    1954-08-06   

             Mg             Al             Si      P  ...    Ta   W  Pt  Au  \
80  1457.666667  106266.333333  308859.666667  468.0  ...  26.0 NaN NaN NaN   

           Hg   Tl         Pb  Bi         Th   U  
80  10.666667  4.0  25.333333 NaN  12.333333 NaN  

[1 rows x 44 columns]


In [17]:
#prep final dataset
final_columns = [
    'lab_sample_id',
    'sample_id',
    'EG',
    'thick_clay',
    'fine_clay',
    'silt',
    'clay',
    'Eq_Hum',
    'atm_1/3',
    'atm_15',
    'CACO3',
    'gypsum',
    'free_iron',
    'organic_carbon',
    'total_N',
    'P205',
    'organic_material',
    'pH_H2O',
    'pH_KCL',
    'Ca++',
    'Mg++',
    'Na+',
    'K+',
    'exchangable_bases_sum',
    'CEC',
    'V',
    'conductivity',
    'soluble_sodium',
    'Min_<0,002',
    'Min_0,05-0,02',
    'Min_0,2-0,05',
    'Min_2-0,2',
    'field_sample_code',
    'Depht',
    'Al',
    'Si',
    'P',
    'S',
    'Cl',
    'Ti',
    'Cr',
    'Mn',
    'Fe',
    'Co',
    'Ni',
    'Cu',
    'Zn',
    'As',
    'Se',
    'Rb',
    'Sr',
    'Zr',
    'Nb',
    'Mo',
    'Cd',
    'Sn',
    'Sb',
    'Ba',
    'Ta',
    'W',
    'Pt',
    'Au',
    'Hg',
    'Tl',
    'Pb',
    'Bi',
    'Th',
    'U',
]

# Add missing columns with NaN values
for col in final_columns:
    if col not in merged1.columns:
        merged1[col] = pd.NA  # or np.nan if you prefer

# Reorder columns
merged1 = merged1[final_columns]

In [18]:
print(analyses_ready[analyses_ready['sample_id'] == 6940])
print(elemental_analyses[elemental_analyses['sample_id'] == 6940])

      lab_sample_id  sample_id minerology_id  EG  thick_clay  fine_clay  silt  \
1361           1362     6940.0          <NA> NaN        31.1       43.8   2.7   

      clay  Eq_Hum  atm_1/3  ...  K+  exchangable_bases_sum  CEC   V  \
1361  22.7    18.9      NaN  ... NaN                    NaN  NaN NaN   

      conductivity  soluble_sodium  Min_<0,002  Min_0,05-0,02  Min_0,2-0,05  \
1361           NaN             0.0         NaN            NaN           NaN   

      Min_2-0,2  
1361        NaN  

[1 rows x 33 columns]
    sample_id Soil Profile field_sample_code Code.1      Depht Sampling Date  \
80     6940.0      P134/59          A-320/59  S-492  1.30-1.70    1954-08-06   

             Mg             Al             Si      P  ...    Ta   W  Pt  Au  \
80  1457.666667  106266.333333  308859.666667  468.0  ...  26.0 NaN NaN NaN   

           Hg   Tl         Pb  Bi         Th   U  
80  10.666667  4.0  25.333333 NaN  12.333333 NaN  

[1 rows x 44 columns]


In [19]:
merged1.head()

,lab_sample_id,sample_id,EG,thick_clay,fine_clay,silt,clay,Eq_Hum,atm_1/3,atm_15,...,Ta,W,Pt,Au,Hg,Tl,Pb,Bi,Th,U
0,1,10999.0,NaN,61.700001,32.8,0.2,5.3,4.6,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,11000.0,NaN,52.799999,35.1,0.7,11.4,6.4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,11001.0,NaN,42.500000,46.2,0.2,11.1,6.3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,11002.0,NaN,42.599998,41.8,0.2,15.4,5.2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,11003.0,NaN,36.799999,47.5,1.2,14.5,7.3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# Define final column list
final_columns = [
    'lab_sample_id', 'sample_id', 'EG', 'thick_clay', 'fine_clay', 'silt', 'clay', 'Eq_Hum', 'atm_1/3', 'atm_15',
    'CACO3', 'gypsum', 'free_iron', 'organic_carbon', 'total_N', 'P205', 'organic_material', 'pH_H2O', 'pH_KCL',
    'Ca++', 'Mg++', 'Na+', 'K+', 'exchangable_bases_sum', 'CEC', 'V', 'conductivity', 'soluble_sodium', 'Min_<0,002',
    'Min_0,05-0,02', 'Min_0,2-0,05', 'Min_2-0,2', 
    'field_sample_code', 'Depth', 'Al', 'Si', 'P', 'S', 'Cl', 'Ti', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn',
    'As', 'Se', 'Rb', 'Sr', 'Zr', 'Nb', 'Mo', 'Cd', 'Sn', 'Sb', 'Ba', 'Ta', 'W', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi',
    'Th', 'U',
]

# Ensure all columns exist in the DataFrame
for col in final_columns:
    if col not in merged1.columns:
        merged1[col] = pd.NA  # or np.nan if preferred

# Reorder DataFrame columns
merged1 = merged1[final_columns]

# Check and print the datatypes of each column
print("Data types of each column in merged1:\n")
print(merged1.dtypes)


Data types of each column in merged1:

lab_sample_id      int64
sample_id        float64
EG               float64
thick_clay       float64
fine_clay        float64
                  ...   
Tl               float64
Pb               float64
Bi               float64
Th               float64
U                float64
Length: 68, dtype: object


In [55]:
merged1.to_csv("/Users/inesschwartz/Desktop/Thesis/tables_clean/analyses_clean.csv", index=False)

### soil profile table

In [56]:
## rename columns
soil_profile = pd.read_excel("/Users/inesschwartz/Desktop/Thesis/tables_soil_database/Perfis_solo.xlsx")

#rename columns
soil_profile.rename(columns={
    'Perfil': 'profile_id',
    'Agrupamento': 'grouping',
    'Pro': 'province',
    'País': 'country',
    'Local': 'location',
    'DATA': 'date',
    'CEP_NOME': 'CEP_NAME',
}, inplace=True)

# Show first few rows
print(soil_profile.head())

  profile_id grouping    REF province country  \
0       1/51     H 62   62.0    Huíla  Angola   
1       1/57      M 9    9.0   Namibe  Angola   
2       1/59    Cb 52   52.0  Cabinda  Angola   
3       1/63   Mj 116  116.0  Malanje  Angola   
4      10/54    H 59'   59.0    Huíla  Angola   

                                               LOCAL  \
0  Estrada Sá da Bandeira - Quijungo a 58,8 km do...   
1  Estrada Moçamedes-Porto Alexandre, a 0,2 km de...   
2  Estrada Buco Zau-Dinge, no cruzamento com o se...   
3                           A 0,7 km a S-SW de Quela   
4  Picada que parte da estrada Vila Paiva Couceir...   

                 DESCRITOR1            DESCRITOR2             DESCRITOR3  \
0    J. V. Botelho da Costa      R. Pinto Ricardo                     AA   
1           E. M. S. Câmara      R. Pinto Ricardo                    NaN   
2           E. M. S. Câmara      R. Pinto Ricardo  J. A. Castanho Póvoas   
3  F. A. Milho da Conceição        J. F. R. Matos       R. Pinto

In [57]:
# Drop unnecessary columns
soil_profile_cleaning1 = soil_profile.drop(columns=[
    'grouping', 'REF', 'province', 'country', 'LOCAL', 'DESCRITOR1', 'DESCRITOR2', 'DESCRITOR3', 'CEP_GR'
])


soil_profile_cleaning2 = soil_profile.drop(columns=[
    'grouping', 'REF', 'province', 'country', 'LOCAL', 'DESCRITOR1', 'DESCRITOR2', 'DESCRITOR3', 'date', 'CEP_GR'
])
# Add missing columns
soil_profile_cleaning2['site_info_id'] = pd.NA
soil_profile_cleaning2['sample_id'] = pd.NA 
soil_profile_cleaning2['soil_type_id'] = pd.NA  

soil_profile_cleaning2['site_info_id'] = soil_profile_cleaning2['site_info_id'].astype(str)



# Reorder columns to match SAMPLES schema
profile_clean = soil_profile_cleaning2[[
    'profile_id',
    'site_info_id',
    'sample_id',
    'soil_type_id'
]]

# Preview the result
profile_clean.head()
#profile_clean.to_csv("/Users/inesschwartz/Desktop/Thesis/tables_clean/profile_clean.csv", index=False)

,profile_id,site_info_id,sample_id,soil_type_id
0,1/51,<NA>,<NA>,<NA>
1,1/57,<NA>,<NA>,<NA>
2,1/59,<NA>,<NA>,<NA>
3,1/63,<NA>,<NA>,<NA>
4,10/54,<NA>,<NA>,<NA>


In [58]:
soil_profile_cleaning1 = soil_profile_cleaning1[[
    'profile_id',
    'date'
]]

soil_profile_cleaning1.head()


,profile_id,date
0,1/51,1951-08-14
1,1/57,1957-06-26
2,1/59,1959-07-07
3,1/63,1963-06-25
4,10/54,1954-07-22


In [59]:

# Check and print the datatypes of each column
print("Data types of each column in profile_clean:\n")
print(profile_clean.dtypes)


Data types of each column in profile_clean:

profile_id      object
site_info_id    object
sample_id       object
soil_type_id    object
dtype: object


## Morphology Horizon

In [60]:
## rename columns
#rename columns
morphology.rename(columns={
    'Morfo_id':'horizon_id',
    'Amostra': 'sample_id',
    'Perfil': 'profile_id',
    'CM':'horizon_layer',
    'Limite Superior': 'upper_depth',
    'Limite inferior': 'lower_depth',
    'Grau de humidade': 'moisture_degree',
    'Quantidade de raízes': 'root_quantity',
    'Diâmetro de raízes': 'root_diameter',
    'Textura': 'texture',
    'Tipo de estrutura': 'structure_type',
    'Classes de estrutura': 'structure_class',
    'Grau de estrutura': 'structure_degree',
    'Diâmetro de poros': 'pore_diameter',
    'Quantidade de poros': 'pore_quantity',
    'Forma de poros': 'pore_shape',
    'Cor (s)': 'dry_color_name',
    'Matiz (s)': 'dry_hue',
    'Valor (s)':'dry_value',
    'Croma (s)': 'dry_chroma',
    'Cor (h)': 'moist_color_name',
    'Matiz (h)': 'moist_hue',
    'Valor (h)': 'moist_value',
    'Croma (h)': 'moist_chroma',
    'Compacidade':'compaction',
    'Dureza': 'durability'
}, inplace=True)

# Drop unnecessary columns
morphology_cleaning = morphology.drop(columns=[
    'ID1', 'Agrupamento', 'REF', 'Pro', 'Observaçoes', 'Horizonte de diagnóstico', 'Propriedade de diagnóstico', 'Nitidez do limite', 'Designação do horizonte', 'Observaçoes', 'Confirmar', 'Adesividade', 'Plasticidade', 'Efervescência com HCl', 'Friabilidade', 'Orientação das Fendas', 'Largura das fendas', 'Quantidade de fendas'
])


#drop accents
import unicodedata
def remove_accents(text):
    if isinstance(text, str):
        # Normalize and remove diacritics
        text = unicodedata.normalize('NFKD', text)
        text = ''.join(c for c in text if not unicodedata.combining(c))
        return text
    return text

# Apply to all cells in the DataFrame
morphology_cleaning = morphology_cleaning.applymap(remove_accents)

morphology_cleaning = morphology_cleaning.fillna('null')



# Reorder columns to match SAMPLES schema
morphology_clean = morphology_cleaning[[
    'horizon_id',
    'sample_id',
    'profile_id',
    'horizon_layer',
    'upper_depth',
    'lower_depth',
    'moisture_degree',
    'root_quantity',
    'root_diameter',
    'texture',
    'structure_type',
    'structure_class',
    'structure_degree',
    'pore_diameter',
    'pore_quantity',
    'pore_shape',
    'dry_color_name',
    'dry_hue',
    'dry_value',
    'dry_chroma',
    'moist_color_name',
    'moist_hue',
    'moist_value',
    'moist_chroma',
    'compaction',
    'durability'
]]

#morphology_clean.head()
#morphology_clean.to_csv("/Users/inesschwartz/Desktop/Thesis/tables_clean/morphology_horizon_clean.csv", index=False)

# Show first few rows
morphology_cleaning.head()
morphology_clean.head(15)

morphology_clean.to_csv("/Users/inesschwartz/Desktop/Thesis/tables_clean/morphology_horizon_clean.csv", index=False)

In [61]:

# Check and print the datatypes of each column
print("Data types of each column in profile_clean:\n")
print(morphology_clean.dtypes)


Data types of each column in profile_clean:

horizon_id          object
sample_id           object
profile_id          object
horizon_layer       object
upper_depth         object
lower_depth         object
moisture_degree     object
root_quantity       object
root_diameter       object
texture             object
structure_type      object
structure_class     object
structure_degree    object
pore_diameter       object
pore_quantity       object
pore_shape          object
dry_color_name      object
dry_hue             object
dry_value           object
dry_chroma          object
moist_color_name    object
moist_hue           object
moist_value         object
moist_chroma        object
compaction          object
durability          object
dtype: object


## Soil type

In [62]:
#rename columns
#rename columns
soil_type = soil_profile

soil_type.rename(columns={
    'Perfil': 'profile_id',
    'Agrupamento': 'grouping',
    'Pro': 'province',
    'País': 'country',
    'Local': 'location',
    'DATA': 'date',
    'CEP_NOME': 'CEP_NAME',
}, inplace=True)

# Drop unnecessary columns
soil_type_cleaning = soil_type.drop(columns=[
    'REF', 'province', 'country', 'LOCAL', 'DESCRITOR1', 'DESCRITOR2', 'DESCRITOR3', 'date', 'Fase', 'D_INSERÇAO', 'Publicação', 'WRB_old', 'Missão'
])
# Add a new Primary Key ID column starting from 1
soil_type_cleaning.insert(0, 'soil_type_id', range(1, len(soil_type_cleaning) + 1))

#drop accents
import unicodedata
def remove_accents(text):
    if isinstance(text, str):
        # Normalize and remove diacritics
        text = unicodedata.normalize('NFKD', text)
        text = ''.join(c for c in text if not unicodedata.combining(c))
        return text
    return text

# Apply to all cells in the DataFrame
soil_type_cleaning = soil_type_cleaning.applymap(remove_accents)

soil_type_cleaning.head()

soil_type_clean = soil_type_cleaning
soil_type_clean.to_csv("/Users/inesschwartz/Desktop/Thesis/tables_clean/soil_type_clean.csv", index=False)

In [63]:

# Check and print the datatypes of each column
print("Data types of each column in profile_clean:\n")
print(soil_type_clean.dtypes)


Data types of each column in profile_clean:

soil_type_id     int64
profile_id      object
grouping        object
CEP_GR          object
CEP_NAME        object
FAO             object
dtype: object


## Site_information

In [64]:
profile_loc.head()

,ID,PERFIL,X_COORD,Y_COORD,LATITUDE,LTGRAUS,LTMIN,LTSEC,LONGITUDE,LGGRAUS,LGMIN,LGSEC,AGRUPAMENT,PRO,LOCAL,TOPOGRAFIA,ALTITUDE,GEOLOGIA,LITOLOGIA,LITOLOGIA_1954,CL_THORNTH,CL_KOPPEN,PMA,TMA,REG_HÍDRIC,REG_TÉRMIC
0,2770,1/57,12.161278,-15.222598,"-15,20108696",-15.0,12.0,3.9,"12,15307971",12.0,9.0,11.1,M 9,Namibe,"Estrada Moçamedes-Porto Alexandre, a 0,2 km de...",NaN,32.0,NaN,d,NaN,E,NaN,NaN,NaN,NaN,NaN
1,48,1/59,12.575775,-4.866986,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,Cb 52,Cabinda,"Estrada Buco Zau-Dinge, no cruzamento com o se...",D5,NaN,Oendolongo,pp,Sistema do Maiombe,B1,NaN,NaN,NaN,NaN,NaN
2,1618,1/61,15.098840,-11.225411,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,Cuanza Sul,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,881,1/63,17.081955,-9.274587,"-9,27458667755",0.0,0.0,0.0,"17,08195495605",0.0,0.0,0.0,Mj 116,Malanje,"A 0,7 km a S-SW de Quela",D1,1210.0,Karroo,Cs/Cal,Série de Cassanje - T2'T1,B3,Aw,1400-1600,21-22,TUSu,iH
4,1750,1/64,20.788116,-11.568683,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,Moxico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
site_info_cleaning = profile_loc

site_info_cleaning.rename(columns={
    'PERFIL': 'profile_id',
    'X_COORD': 'X_coord',
    'Y_COORD': 'Y_coord'
}, inplace=True)

# Add a new Primary Key ID column starting from 1
site_info_cleaning.insert(0, 'site_info_id', range(1, len(site_info_cleaning) + 1))


# Add missing columns
site_info_cleaning['land_cover_id'] = pd.NA
site_info_cleaning['climate_id'] = pd.NA  
site_info_cleaning['geology_id'] = pd.NA
site_info_cleaning['topo_feature_id'] = pd.NA  
site_info_cleaning['sampling_date'] = pd.NA
site_info_cleaning['districts_id'] = pd.NA  


site_info_clean = site_info_cleaning[[
    'site_info_id',
    'profile_id', ##do I add profile ID here to then try to merge date info from soil Perfis_solo table? or is this too complicated?
    'X_coord',
    'Y_coord',
    'land_cover_id',
    'climate_id',
    'geology_id',
    'topo_feature_id',
    'sampling_date',
    'districts_id'
]]

site_info_clean.head()

,site_info_id,profile_id,X_coord,Y_coord,land_cover_id,climate_id,geology_id,topo_feature_id,sampling_date,districts_id
0,1,1/57,12.161278,-15.222598,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2,1/59,12.575775,-4.866986,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,3,1/61,15.098840,-11.225411,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,4,1/63,17.081955,-9.274587,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,5,1/64,20.788116,-11.568683,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [66]:
##merging date info from perfis_solo to site_info table

merged = soil_profile_cleaning1.merge(site_info_clean, on='profile_id', how='left')

merged['site_info_id'] = merged['site_info_id'].astype(str)


pd.set_option('display.max_columns', None)
merged.head()

merged.to_csv("/Users/inesschwartz/Desktop/Thesis/tables_clean/site_info_clean.csv", index=False)



In [67]:
# Check and print the datatypes of each column
print("Data types of each column in profile_clean:\n")
print(merged.dtypes)

Data types of each column in profile_clean:

profile_id                 object
date               datetime64[ns]
site_info_id               object
X_coord                   float64
Y_coord                   float64
land_cover_id              object
climate_id                 object
geology_id                 object
topo_feature_id            object
sampling_date              object
districts_id               object
dtype: object


## Climate table


In [68]:
# Load original Excel data
profile_loc = pd.read_excel("/Users/inesschwartz/Desktop/Thesis/tables_soil_database/Perfis_local.xlsx")

# Create a copy for cleaning
climate_features_cleaning = profile_loc.copy()

# Mapping from climate codes to descriptions
code_to_description = {
    "B1": "Húmido",
    "B2": "Húmido",
    "B3": "Húmido",
    "B4": "Húmido",
    "C1": "Sub-húmido seco",
    "C2": "Sub-húmido chuvoso",
    "D": "Semi-árido",
    "E": "Árido",
    "Aw": "Tropical chuvoso com estação seca no Inverno, de savana",
    "BSw": "Seco de estepe, com chuva predominante no Verão",
    "BWw": "Seco de deserto, com chuva predominante no Verão",
    "Cw": "Mesotérmico húmido com estação seca no Inverno",
    "ARe": "Arídico extremo",
    "ARf": "Arídico fraco",
    "ARt": "Arídico típico",
    "tUDs": "Tempúdico seco",
    "tUSh": "Tempústico húmido",
    "tUSt": "Tempústico típico",
    "TUDs": "Tropúdico seco",
    "TUSa": "Tropústico arídico",
    "TUSu": "Tropústico údico",
    "TUSt": "Tropustico típico",
    "H": "Hipertérmico",
    "iH": "Iso-Hipertérmico",
    "iT": "Iso-Térmico",
    "T": "Térmico"
}


# Replace climate codes with descriptions directly in the two columns
climate_features_cleaning["CL_THORNTH"] = climate_features_cleaning["CL_THORNTH"].replace(code_to_description)
climate_features_cleaning["CL_KOPPEN"] = climate_features_cleaning["CL_KOPPEN"].replace(code_to_description)
climate_features_cleaning["REG_HÍDRIC"] = climate_features_cleaning["REG_HÍDRIC"].replace(code_to_description)
climate_features_cleaning["REG_TÉRMIC"] = climate_features_cleaning["REG_TÉRMIC"].replace(code_to_description)

# Rename columns for clarity
climate_features_cleaning.rename(columns={
    'PERFIL': 'profile_id',
    'CL_THORNTH': 'thornthwaite_climate',
    'CL_KOPPEN': 'koppen_climate',
    'TMA': 'mean_annual_temp',
    'PMA': 'mean_annual_precip',
    'REG_HÍDRIC': 'hydric_regime',
    'REG_TÉRMIC': 'thermal_regime'
}, inplace=True)

# Add primary key column
climate_features_cleaning.insert(0, 'climate_id', range(1, len(climate_features_cleaning) + 1))

# Select and reorder relevant columns
climate_features_clean = climate_features_cleaning[[
    'climate_id',
    'profile_id',  # Yes, keep this to allow merging with soil profile data later
    'mean_annual_temp',
    'mean_annual_precip',
    'koppen_climate',
    'thornthwaite_climate',
    'hydric_regime',
    'thermal_regime'
]]

#drop accents
import unicodedata
def remove_accents(text):
    if isinstance(text, str):
        # Normalize and remove diacritics
        text = unicodedata.normalize('NFKD', text)
        text = ''.join(c for c in text if not unicodedata.combining(c))
        return text
    return text

# Apply to all cells in the DataFrame
climate_features_cleaning = climate_features_cleaning.applymap(remove_accents)

# Preview
climate_features_clean.head()
climate_features_clean.to_csv("/Users/inesschwartz/Desktop/Thesis/tables_clean/climate_features_clean.csv", index=False)





In [69]:
# Check and print the datatypes of each column
print("Data types of each column in profile_clean:\n")
print(climate_features_clean.dtypes)

Data types of each column in profile_clean:

climate_id               int64
profile_id              object
mean_annual_temp        object
mean_annual_precip      object
koppen_climate          object
thornthwaite_climate    object
hydric_regime           object
thermal_regime          object
dtype: object


## Topographic features

In [70]:
import pandas as pd

# Load original Excel data
profile_loc = pd.read_excel("/Users/inesschwartz/Desktop/Thesis/tables_soil_database/Perfis_local.xlsx")

# Create a copy for cleaning
topo_features_cleaning = profile_loc.copy()

# Rename columns for clarity
topo_features_cleaning.rename(columns={
    'PERFIL': 'profile_id',
    'TOPOGRAFIA': 'slope_code',
    'ALTITUDE': 'altitude',
}, inplace=True)

# Add missing columns
topo_features_cleaning['aspect'] = pd.NA  
topo_features_cleaning['land_surface_temp'] = pd.NA
topo_features_cleaning['dem_elevation'] = pd.NA  

# Add primary key column
topo_features_cleaning.insert(0, 'topo_features_id', range(1, len(topo_features_cleaning) + 1))

# Create slope class mapping dictionary
slope_code_to_description = {
    "D1": "Plano (Declives < 2%)",
    "D2": "Ondulado muito suave (Declives > 2% e < 3%)",
    "D3": "Ondulado suave (Declives > 3% e < 5%)",
    "D4": "Ondulado (Declives > 5% e < 8%)",
    "D5": "Acidentado (Declives > 8% e < 15%)",
    "D6": "Escarpado (Declives >15% e < 30%)",
    "D7": "Montanhoso (Declives > 30%)"
}

# Create a mapping DataFrame for slope classes
slope_classes_df = pd.DataFrame([
    {"slope_code": code, "slope_description": desc}
    for code, desc in slope_code_to_description.items()
])

# Save mapping table to CSV
slope_classes_df.to_csv("/Users/inesschwartz/Desktop/Thesis/tables_clean/slope_classes_mapping.csv", index=False)

# Final cleaned topo features table (referencing slope_code, not description)
topo_features_clean = topo_features_cleaning[[
    'topo_features_id',
    'profile_id', 
    'slope_code',
    'altitude',
    'aspect',
    'land_surface_temp',
    'dem_elevation'
]]

# Save cleaned topo features table
topo_features_clean.to_csv("/Users/inesschwartz/Desktop/Thesis/tables_clean/topo_features_clean.csv", index=False)

# Preview
topo_features_clean.head()


,topo_features_id,profile_id,slope_code,altitude,aspect,land_surface_temp,dem_elevation
0,1,1/57,NaN,32.0,<NA>,<NA>,<NA>
1,2,1/59,D5,NaN,<NA>,<NA>,<NA>
2,3,1/61,NaN,NaN,<NA>,<NA>,<NA>
3,4,1/63,D1,1210.0,<NA>,<NA>,<NA>
4,5,1/64,NaN,NaN,<NA>,<NA>,<NA>


In [71]:
# Check and print the datatypes of each column
print("Data types of each column in profile_clean:\n")
print(topo_features_clean.dtypes)

Data types of each column in profile_clean:

topo_features_id       int64
profile_id            object
slope_code            object
altitude             float64
aspect                object
land_surface_temp     object
dem_elevation         object
dtype: object


## Geological Features

In [72]:
import pandas as pd

# Load cleaned geo features data
geo_features_cleaning = pd.read_excel("/Users/inesschwartz/Desktop/Thesis/tables_soil_database/Perfis_local.xlsx")

# Rename columns
geo_features_cleaning.rename(columns={
    'PERFIL': 'profile_id',
    'GEOLOGIA': 'geology_id',
    'LITOLOGIA': 'lithology_id',
    'LITOLOGIA_1954': 'lithology_1954_id',
}, inplace=True)

# Add primary key column
geo_features_cleaning.insert(0, 'geo_features_id', range(1, len(geo_features_cleaning) + 1))

# Final normalized geo_features table (with codes as foreign keys)
geo_features_clean = geo_features_cleaning[[
    'geo_features_id',
    'profile_id',
    'geology_id',
    'lithology_id',
    'lithology_1954_id'
]]

# Save the cleaned table
geo_features_clean.to_csv("/Users/inesschwartz/Desktop/Thesis/tables_clean/geo_features_clean.csv", index=False)

#preview
geo_features_clean.head()

,geo_features_id,profile_id,geology_id,lithology_id,lithology_1954_id
0,1,1/57,NaN,d,NaN
1,2,1/59,Oendolongo,pp,Sistema do Maiombe
2,3,1/61,NaN,NaN,NaN
3,4,1/63,Karroo,Cs/Cal,Série de Cassanje - T2'T1
4,5,1/64,NaN,NaN,NaN


In [73]:
## mapping tables
# Mappings for geology
geology_mapping = {
    "Kalahari": "Sistema do Kalahari",
    "Superficiais": "Formações Superficiais",
    "Karroo": "Sistema do Karroo",
    "Bembe": "Sistema do Bembe",
    "Oendolongo": "Sistema do Oendolongo",
    "Base": "Complexo de base",
    "Proterozóico": "Proterozóico",
    "Pleistocénico": "Pleistocénico",
    "Terciário": "Terciário (médio e inferior)",
    "TQ": "Quaternário e Terciário superior",
    "Cretácio": "",  # no description provided
    "RPKS": "Recente Plistocénico e Kalahari Superior"
}

# Mappings for lithology_1954
lithology_1954_mapping = {
    "γ": "Granitos, Granodioritos e Quartzodioritos",
    "PL": "Xistos, metaquartzitos, conglomerados, arcoses, ect.",
    "λ": "Rochas eruptivas indeterminadas",
    "δp": "Doleritos, doleritos pigeoníticos",
    "δab": "Diabases, diabases albito-cloriticas",
    "ε": "Noritos, gabros e peridotitos",
    "JK": "Composto de conglomerados, areias, cascalhos do Kalahari",
    "C": "Série Xisto - calcária",
    "Cal": "Sedimentos arenosos não consolidados",
    "K": "Série xisto - gresosa",
    "RT": "Não diferenciado",
    "σ": "Sienitos, sienitos nefelínicos",
    "Q": "Depósitos fossilíferos",
    "CS": "Grande conglomerado e série de Mwashya"
}

# Mappings for lithology
lithology_mapping = {
    "a": "Rochas arenáceas consolidadas",
    "aq": "Grés quartzíticos do Oendolongo",
    "b": "Rochas eruptivas básicas",
    "c": "Rochas sedimentares consolidadas calcárias",
    "c'": "Rochas sedimentares não consolidadas calcárias",
    "cg": "Rochas cristalofílicas argiláceas",
    "d": "Sedimentos não consolidados de origem marinha",
    "dc": "Depósitos coluvionares",
    "dr": "Diorítos",
    "e": "Rochas sedimentares consolidadas não calcárias",
    "g": "Rochas argiláceas consolidadas não calcárias",
    "g'": "Rochas argiláceas não consolidadas não calcárias",
    "g''": "Rochas cristalinas pouco micas em quartzo",
    "gp": "Rochas do  complexo gabro-plagioclastíco",
    "k": "Sedimentos não consolidados grosseiros do Kalahari",
    "m": "Rochas sedimentares não consolidadas calco-gipsíferas",
    "m'": "Depósitos coluvionares margosos",
    "mm": "Materiais mistos",
    "n": "Sedimentos não consolidados de origem continental",
    "nd": "não descrito",
    "pp": "Sedimentos não consolidados grosseiros plio-plistocénicos",
    "q": "Rochas cristalinas quartzíferas",
    "q'": "Materiais redistribuídos provenientes de desagregação rochas crist. quartzíferas",
    "qf": "Quartzitos ferruginosos do Oendolongo",
    "r": "Sedimentos grosseiros não especificados",
    "s": "Sienitos",
    "sx": "Formações (ou rochas) sedimentares não especificadas",
    "sx1": "Rochas sedimentares consolidadas com e sem calcário",
    "sx2": "Rochas sedimentares consolidadas",
    "v": "Materiais vulcânicos",
    "v'": "Rochas do complexo alcalino e/ou carboatítico",
    "x": "Rochas consolidadas não especificadas",
    "xm": "Xistos metamórficos",
    "xq": "Rochas cristalinas não especificadas",
    "z": "Rochas metassedimentares"
}

# Save each mapping as a DataFrame
pd.DataFrame([
    {"geology_code": k, "geology_description": v}
    for k, v in geology_mapping.items()
]).to_csv("/Users/inesschwartz/Desktop/Thesis/tables_clean/geology_mapping.csv", index=False)

pd.DataFrame([
    {"lithology_code": k, "lithology_description": v}
    for k, v in lithology_mapping.items()
]).to_csv("/Users/inesschwartz/Desktop/Thesis/tables_clean/lithology_mapping.csv", index=False)

pd.DataFrame([
    {"lithology_1954_code": k, "lithology_1954_description": v}
    for k, v in lithology_1954_mapping.items()
]).to_csv("/Users/inesschwartz/Desktop/Thesis/tables_clean/lithology1954_mapping.csv", index=False)


In [74]:
# Check and print the datatypes of each column
print("Data types of each column in lithology_mapping:\n")
print(geo_features_clean.dtypes)

Data types of each column in lithology_mapping:

geo_features_id       int64
profile_id           object
geology_id           object
lithology_id         object
lithology_1954_id    object
dtype: object


## Districts

In [75]:
samples_filled.head()

,sample_id,site_info_id,year,profile_id,campaign,country,district,sample_sifted,Província,sample_not_sifted,shelf,room
0,630,172,1946.0,139,MA/46,Angola,Huambo,X,Huambo,null,1,22
1,631,173,1946.0,139,MA/46,Angola,Huambo,X,Huambo,null,1,22
2,632,174,1946.0,139,MA/46,Angola,Huambo,X,Huambo,null,1,22
3,633,175,1946.0,139,MA/46,Angola,Huambo,X,Huambo,null,1,22
4,687,1034,1946.0,208,MA/46,Angola,Huambo,X,Huambo,null,1,22


In [76]:
# Final normalized districts table (with codes as foreign keys)
district_cleaning = samples

# Add primary key column
district_cleaning.insert(0, 'district_id', range(1, len(district_cleaning) + 1))

district_clean = district_cleaning[[
    'district_id',
    'sample_id',
    'profile_id',
    'district'
]]

district_clean['sample_id'] = district_clean['sample_id'].astype(float)


district_clean.to_csv("/Users/inesschwartz/Desktop/Thesis/tables_clean/district_clean.csv", index=False)
#preview:
district_clean.head()

/var/folders/tp/79mdnyy56_xc3g1jvp9wf4_80000gn/T/ipykernel_12211/3700924969.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  district_clean['sample_id'] = district_clean['sample_id'].astype(float)


,district_id,sample_id,profile_id,district
0,1,630.0,139,Huambo
1,2,631.0,139,Huambo
2,3,632.0,139,Huambo
3,4,633.0,139,Huambo
4,5,687.0,208,Huambo


## Minerology_info tables

In [77]:
# Load cleaned minerology features data
minerology_cleaning = pd.read_excel("/Users/inesschwartz/Desktop/Thesis/tables_soil_database/Perfis_local.xlsx")

# Rename columns
geo_features_cleaning.rename(columns={
    'PERFIL': 'profile_id',
    'GEOLOGIA': 'geology_id',
    'LITOLOGIA': 'lithology_id',
    'LITOLOGIA_1954': 'lithology_1954_id',
}, inplace=True)

# Add primary key column
#geo_features_cleaning.insert(0, 'geo_features_id', range(1, len(geo_features_cleaning) + 1))

# Final normalized geo_features table (with codes as foreign keys)
geo_features_clean = geo_features_cleaning[[
    'geo_features_id',
    'profile_id',
    'geology_id',
    'lithology_id',
    'lithology_1954_id'
]]

geo_features_clean.head()

,geo_features_id,profile_id,geology_id,lithology_id,lithology_1954_id
0,1,1/57,NaN,d,NaN
1,2,1/59,Oendolongo,pp,Sistema do Maiombe
2,3,1/61,NaN,NaN,NaN
3,4,1/63,Karroo,Cs/Cal,Série de Cassanje - T2'T1
4,5,1/64,NaN,NaN,NaN


## Checking csv data types

In [78]:
import pandas as pd
import os

# Set the path to your folder containing the 10 CSV files
csv_folder = "/Users/inesschwartz/Desktop/Thesis/tables_clean"  

# List all CSV files in the folder
csv_files = [f for f in os.listdir(csv_folder) if f.endswith(".csv")]

# Loop through each file and display column data types
for file in csv_files:
    file_path = os.path.join(csv_folder, file)
    print(f"\n📄 File: {file}")
    
    try:
        df = pd.read_csv(file_path)
        print(df.dtypes)
    except Exception as e:
        print(f"⚠️ Error reading {file}: {e}")


📄 File: district_clean.csv
district_id      int64
sample_id      float64
profile_id      object
district        object
dtype: object

📄 File: soil_type_clean.csv
soil_type_id     int64
profile_id      object
grouping        object
CEP_GR          object
CEP_NAME        object
FAO             object
dtype: object

📄 File: analyses_clean.csv
lab_sample_id      int64
sample_id        float64
EG               float64
thick_clay       float64
fine_clay        float64
                  ...   
Tl               float64
Pb               float64
Bi               float64
Th               float64
U                float64
Length: 68, dtype: object

📄 File: climate_features_clean.csv
climate_id               int64
profile_id              object
mean_annual_temp        object
mean_annual_precip      object
koppen_climate          object
thornthwaite_climate    object
hydric_regime           object
thermal_regime          object
dtype: object

📄 File: analyses_clean_check2.csv
lab_sample_id      int6